In [1]:
# Imports
import pandas as pd
import numpy as np
import cx_Oracle
import pandas as pd
from datetime import date, timedelta
import urllib3
from unidecode import unidecode
from oauth2client.service_account import ServiceAccountCredentials
from IPython.core.magic import (Magics, magics_class, cell_magic)
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

# Plotting
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})

# Pre processing and manipulation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import joblib

# metrics and hyperparameter optimization
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
from skopt import dummy_minimize
from skopt import gp_minimize
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# models
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn import svm, tree
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm
from sklearn.ensemble import RandomForestRegressor

In [2]:
cx_Oracle.init_oracle_client(lib_dir="C:\oracle\instantclient_19_8", config_dir="C:\oracle\instantclient_19_8")

In [7]:
pw = ""
conn = cx_Oracle.connect("", pw, "", encoding="UTF-8")

In [8]:
%%time
sql_pedidos = '''SELECT V_ORDER_ALL.LOG_SEARCH_ID "Pesquisa", V_ORDER_ALL.ID_ORDER "Pedido", V_ORDER_ALL.CUSTOMER_ID "ID Cliente", V_ORDER_ALL.CUSTOMER "Cliente", V_ORDER_ALL.TELEPHONE_CUSTOMER "Telefone", V_ORDER_ALL.ORDER_DATE "Data", V_ORDER_ALL.MARKET_RESELLER "Mercado", V_ORDER_ALL.CORE_RESELLER "Núcleo", V_ORDER_ALL.MICRO_MARKET_RESELLER "Micro Mercado", V_ORDER_ALL.CNPJ_RESELLER "CNPJ", V_ORDER_ALL.COMPANY_NAME_RESELLER "Nome Fantasia", V_ORDER_ALL.ORDER_ADDRESS "Endereço", V_ORDER_ALL.ORDER_ADDRESS_NUMBER "Nº", V_ORDER_ALL.ORDER_ADDRESS_COMPLEMENT "Complemento", V_ORDER_ALL.ORDER_ADDRESS_NEIGHBORHOOD "Bairro", V_ORDER_ALL.ORDER_ADDRESS_REFERENCE "Referência", V_ORDER_ALL.ZIPCODE_CUSTOMER "CEP", V_ORDER_ALL.CITY_RESELLER "Cidade", V_ORDER_ALL.UF_RESELLER "UF", V_ORDER_ALL.CHANNEL "Canal", V_ORDER_ALL.ORDER_VOUCHER_PARTNER "Código Parceiro", V_ORDER_ALL.SOURCE_ACCESS "Origem Acesso", V_ORDER_ALL.SOURCE "Modo", V_ORDER_ALL.ORDER_STATUS_DESCR "Status", TB_ORDER.DELIVERY_DATE "Data Entrega", V_ORDER_ALL.ORDER_RAITING "Avaliação", V_ORDER_ALL.PRODUCT "Produto", V_ORDER_ALL.PRODUCT_PRICE "Preço", V_ORDER_ALL.PRODUCT_QUANTITY "Quantidade", V_ORDER_ALL.ORDER_PACKING "Vasilhame", V_ORDER_ALL.ORDER_PACKING_COST "Preço Vasilhame", V_ORDER_ALL.ORDER_PAYMENT_METHOD "Forma Pagto", V_ORDER_ALL.ORDER_TOTAL "Valor Pedido", V_ORDER_ALL.ORDER_FEE "Taxa uso Plataforma (%)", V_ORDER_ALL.ORDER_FEE_TOTAL "Taxa uso Plataforma (R$)", V_ORDER_ALL.ORDER_VOUCHER_CODE "Vale Gás", V_ORDER_ALL.ORDER_VOUCHER_TYPE "Tipo Vale Gás", V_ORDER_ALL.COUPON "Cupom", V_ORDER_ALL.COUPON_PARTNER "Cupom Parceria", V_ORDER_ALL.COUPON_PARTNER_NAME "Cupom Nome Parceiro", V_ORDER_ALL.COUPON_PARTNER_COST "Cupom Custo Parceiro", V_ORDER_ALL.ORDER_SHIP_PROMISSE_TIME "Tempo Previsto Entrega", V_ORDER_ALL.NAME_DELIVERYMAN "Entregador", V_ORDER_ALL.CANCELLATION_REASON "Motivo Cancelamento", V_ORDER_ALL.CANCELLATION_REASON_TYPE "Justificativa Cancelamento", V_ORDER_ALL.ORDER_CANCEL_DATE "Data Cancelamento", V_ORDER_ALL.ORDER_REDIRECT_RESELLER "Pedido Remanejado", V_ORDER_ALL.ORDER_CANCEL_USER "Usuário Cancelamento", V_ORDER_ALL.ORDER_ID_NEW_REDIRECT "Novo Pedido", V_ORDER_ALL.SESSION_ID "Sessão", V_ORDER_ALL.ORDER_CANCEL_SUB_REASON "Sub Motivo Cancelamento", V_ORDER_ALL.COMPANY_RESELLER "Razão Social", V_ORDER_ALL.ORDER_PAYMENT_ID "Payment ID"
FROM V_ORDER_ALL
    INNER JOIN TB_ORDER
    ON V_ORDER_ALL.ID_ORDER = TB_ORDER.ID_ORDER
WHERE ORDER_DATE > TO_DATE('2020-01-01','YYYY-MM-DD')
AND ORDER_DATE < TO_DATE('2021-01-01','YYYY-MM-DD')
'''

sql_revendas = '''SELECT TB_RESELLER.CNPJ, V_RESELLER_LIST.LINK_PRODUCT, V_RESELLER_LIST.LINK_AREA, TB_RESELLER.STATUS, TB_RESELLER.LATITUDE, TB_RESELLER.LONGITUDE, TB_RESELLER.INTERNAL_OPERATION "OP INTERNA", TB_RESELLER.MARKET_ID, TB_RESELLER.MARKET_CORE_ID, TB_RESELLER.MICRO_MARKET_ID, TB_RESELLER.CITY_ID, TB_RESELLER.CURRENT_FEE, TB_RESELLER.VALIDATE_ONLINE_CODE, TB_RESELLER.PREMIUM_VALUE, TB_RESELLER.RAITING, TB_RESELLER.IS_OPEN,HOUR_START, TB_RESELLER.HOUR_END, TB_RESELLER.HOUR_START_VACANCY, TB_RESELLER.HOUR_END_VACANCY, TB_RESELLER.RECEIVE_SMS, TB_RESELLER.MARKETPLACE, TB_RESELLER.DIRECT_SALE, TB_RESELLER.ENABLE_SCHEDULE  
FROM TB_RESELLER
    INNER JOIN V_RESELLER_LIST
    ON TB_RESELLER.CNPJ = V_RESELLER_LIST.CNPJ
'''

df_pedidos = pd.read_sql_query(sql_pedidos, conn)
df_revendas = pd.read_sql_query(sql_revendas, conn)

Wall time: 2min 32s


In [9]:
def to_datetime():
    df_pedidos['Data'] = pd.to_datetime(df_pedidos['Data'], dayfirst=True)
    
def new_data_hora():
    df_pedidos['Data Ajustada'] = [d.date() for d in df_pedidos['Data']]
    df_pedidos['Hora'] = [d.time() for d in df_pedidos['Data']]

In [10]:
to_datetime()
new_data_hora()

In [11]:
df_pedidos['Mês'] = pd.DatetimeIndex(df_pedidos['Data Ajustada']).month

In [12]:
jon = df_pedidos.groupby(['CNPJ'])
jon_mes = jon['Mês'].nunique()
df_jon_mes = pd.DataFrame(jon_mes)
df_jon_mes['Mês'].value_counts()

9     196
7     159
8     147
2     134
6     119
1      92
4      86
5      79
3      68
12     11
11      1
10      1
Name: Mês, dtype: int64

In [1]:
jon_valor = jon.aggregate({"Valor Pedido":sum})
df_jon_val = pd.DataFrame(jon_valor)
df_jon_val

NameError: name 'jon' is not defined

In [2]:
#DISCLOSED
df_jon_geral = df_jon_val.merge(df_jon_mes, on='CNPJ')
df_jon_geral['Média Mês'] = df_jon_geral['Valor Pedido'] / df_jon_geral['Mês']
df_jon_geral

NameError: name 'df_jon_val' is not defined

In [3]:
#DISCLOSED
df_rev_work = df_revendas.merge(df_jon_geral, on='CNPJ')
df_rev_work

NameError: name 'df_revendas' is not defined

In [17]:
pd.options.display.max_columns = 500

In [18]:
df_rev_work.columns

Index(['CNPJ', 'LINK_PRODUCT', 'LINK_AREA', 'STATUS', 'LATITUDE', 'LONGITUDE',
       'OP INTERNA', 'MARKET_ID', 'MARKET_CORE_ID', 'MICRO_MARKET_ID',
       'CITY_ID', 'CURRENT_FEE', 'VALIDATE_ONLINE_CODE', 'PREMIUM_VALUE',
       'RAITING', 'IS_OPEN', 'HOUR_START', 'HOUR_END', 'HOUR_START_VACANCY',
       'HOUR_END_VACANCY', 'RECEIVE_SMS', 'MARKETPLACE', 'DIRECT_SALE',
       'ENABLE_SCHEDULE', 'Valor Pedido', 'Mês', 'Média Mês'],
      dtype='object')

In [19]:
#df_rev_work2 = df_rev_work.drop(columns=['Nome Fantasia', 'ID', 'Endereço', 'Número', 'Grupos', 'Telefone Celular', 'Telefone Fixo', 'Email', 'Código do Cliente', 'Código do Endereço', 'ID Revenda Backup', 'Nome Revenda Backup', 'Total Pedidos', 'Pedidos Aguardando Pagamento', 'Pedidos Agendado', 'Pedidos Aberto', 'Pedidos Andamento', 'Pedidos Cancelado Revenda', 'Pedidos Cancelado Cliente', 'Pedidos Cancelado Outros', 'Pedidos Entregue', 'Tempo Aceite', 'Tempo Andamento', 'Tempo Entrega', 'Tempo Cancelamento', 'Início Cálculo Performance', 'Data Base', 'Dia do mês para geração faturamento', 'Método de Pagamento Faturamento', 'Condição de Pagamento', 'Taxa Administrador (%)', 'Tipo de Ajuste da Taxa', 'Valor de Tarifa (R$)', 'Tipo de Ajuste da Tarifa', 'ID Braspag', 'Razão Social','Taxa de Serviço',])
df_rev_work2 = df_rev_work

In [20]:
df_rev_work.columns

Index(['CNPJ', 'LINK_PRODUCT', 'LINK_AREA', 'STATUS', 'LATITUDE', 'LONGITUDE',
       'OP INTERNA', 'MARKET_ID', 'MARKET_CORE_ID', 'MICRO_MARKET_ID',
       'CITY_ID', 'CURRENT_FEE', 'VALIDATE_ONLINE_CODE', 'PREMIUM_VALUE',
       'RAITING', 'IS_OPEN', 'HOUR_START', 'HOUR_END', 'HOUR_START_VACANCY',
       'HOUR_END_VACANCY', 'RECEIVE_SMS', 'MARKETPLACE', 'DIRECT_SALE',
       'ENABLE_SCHEDULE', 'Valor Pedido', 'Mês', 'Média Mês'],
      dtype='object')

In [1]:
## UNFORTUNATELY UNABLE TO EXECUTE FROM HERE ON DUE TO ACCESS ISSUES
## BUT THE METODOLOGY IS THERE, MANAGED TO GET UP TO 0.72 LAST ROLL

In [ ]:
def jackson(x):
    if x < 500:
        return 1
    elif x < 1000:
        return 2
    elif x < 2000:
        return 3
    elif x < 3000:
        return 4
    elif x < 5000:
        return 5
    elif x < 10000:
        return 6
    elif x < 20000:
        return 7
    else:
        return 8

In [ ]:
df_rev_work['class'] = df_rev_work['Média Mês'].apply(jackson)
df_rev_work['class'].value_counts()

In [ ]:
df_rev_work=df_rev_work[df_rev_work['Média Mês'] != 0]

In [ ]:
#df_rev_local = df_rev_work[['CEP', 'Canais', 'UF', 'Cidade', 'Bairro', 'Média Mês']]
df_rev_local = df_rev_work[['CNPJ', 'LINK_PRODUCT', 'LINK_AREA', 'STATUS', 'LATITUDE', 'LONGITUDE', 'OP INTERNA', 'MARKET_ID',
       'MARKET_CORE_ID', 'MICRO_MARKET_ID', 'CITY_ID', 'CURRENT_FEE',
       'VALIDATE_ONLINE_CODE', 'PREMIUM_VALUE', 'RAITING', 'IS_OPEN',
       'HOUR_START', 'HOUR_END', 'RECEIVE_SMS', 'MARKETPLACE', 'DIRECT_SALE', 'ENABLE_SCHEDULE', 'class']]

In [ ]:
#X = df_rev_local.iloc[:,0].values
#ohenc = OneHotEncoder()
#jaxle = X.reshape(-1, 1)
#jaxre = re.fit_transform(jaxle).toarray()
#bobby = pd.DataFrame(jaxre)
#bobby.columns = re.get_feature_names()

In [ ]:
#jonny = ohenc.fit_transform(df_rev_local)

In [ ]:
df_rev_local.info()

In [ ]:
df_rev_local.info()

In [ ]:
df_rev_local['class'].value_counts()

In [ ]:
df_rev_local.head()

In [ ]:
df_jonny = pd.get_dummies(df_rev_local[['STATUS', 'OP INTERNA', 'VALIDATE_ONLINE_CODE', 'IS_OPEN',
       'HOUR_START', 'HOUR_END',
       'RECEIVE_SMS', 'MARKETPLACE', 'DIRECT_SALE', 'ENABLE_SCHEDULE']])

In [ ]:
df_jonny

In [ ]:
df_rev_localxxx = pd.concat([df_rev_local, df_jonny], axis=1)

In [ ]:
df_rev_localxxx

In [ ]:
df_rev_local_final = df_rev_localxxx.drop(columns=['CNPJ', 'STATUS', 'OP INTERNA', 'VALIDATE_ONLINE_CODE', 'IS_OPEN',
       'HOUR_START', 'HOUR_END', 'RECEIVE_SMS', 'MARKETPLACE', 'DIRECT_SALE', 'ENABLE_SCHEDULE'])

In [ ]:
df_rev_local_final['class'].value_counts()

In [ ]:
cols=pd.Series(df_rev_local_final.columns)
for dup in df_rev_local_final.columns[df_rev_local_final.columns.duplicated(keep=False)]: 
    cols[df_rev_local_final.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(df_rev_local_final.columns.get_loc(dup).sum())]
                                    )
df_rev_local_final.columns=cols

In [ ]:
df_rev_local_final2 = df_rev_local_final.loc[:,~df_rev_local_final.columns.duplicated()]

In [ ]:
import re
df_rev_local_final2 = df_rev_local_final.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
df_rev_local_final2 = df_rev_local_final2.loc[:,~df_rev_local_final2.columns.duplicated()]

In [ ]:
df_rev_local_final3 = df_rev_local_final2.dropna(subset=['LATITUDE', 'MARKET_ID'])
#df_rev_local_final3 = df_rev_local_final3[df_rev_local_final3['MdiaMs'] <= 50000]

In [ ]:
df_rev_local_final3['PREMIUM_VALUE'] = df_rev_local_final3['PREMIUM_VALUE'].fillna(0)
df_rev_local_final3['CURRENT_FEE'] = df_rev_local_final3['CURRENT_FEE'].fillna(0)

In [ ]:
df_rev_local_final3['PREMIUM_VALUE'].isna().value_counts()

In [ ]:
X = df_rev_local_final3.drop(columns=['class'])
y = df_rev_local_final3['class'] #.astype('float64')

In [ ]:
y.plot.hist(bins=50)

In [ ]:
y_log = np.log(y)
y_log.plot.hist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

In [ ]:
X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X, y_log, test_size=0.50, random_state=42)

In [ ]:
y_log_test.plot.hist()

In [ ]:
X

In [ ]:
#comparando os modelos para verificar qual utilizar
#isso demorou MUITO pra rodar
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# criando uma lista com todos os modelos
classifiers = [
    #KNeighborsClassifier(3),
    #GaussianNB(),
    #LogisticRegression(),
    #SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    LGBMClassifier(),
    xgb.XGBClassifier()]

In [ ]:
for clf in classifiers:
    # ajustando o modelo
    clf.fit(X_train, y_train)
    # armazenando o nome do modelo
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados
    print('****Results****')
    # fazendo predições
    # calculando as métricas
    y_pred = clf.predict(X_test)
    # imprimindo as métricas
    print("Score:", clf.score(X_test, y_test))
    #print("Precision:", metrics.precision_score(y_test, y_pred))
    #print("Recall:", metrics.recall_score(y_test, y_pred))

In [ ]:
for clf in classifiers:
    # ajustando o modelo
    clf.fit(X_log_train, y_log_train)
    # armazenando o nome do modelo
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados
    print('****Results****')
    # fazendo predições
    # calculando as métricas
    y_pred = clf.predict(X_log_test)
    # imprimindo as métricas
    print("Score:", clf.score(X_log_test, y_log_test))
    #print("Precision:", metrics.precision_score(y_test, y_pred))
    #print("Recall:", metrics.recall_score(y_test, y_pred))

# TUNANDO O GRADIENT BOOST

In [ ]:
def treinar_modelo_gradient(params):
    #random_state = params[0]
    #learning_rate = params[0]
    #n_estimators = params[0]
    subsample = params[0]
    #min_samples_split = params[0]
    #max_features = params[0]
    #min_weight_fraction_leaf = params[0]
    #max_depth = params[0]
    
    
    print(params, '\n')
    
    #learning_rate=0.45389327382098293 ou 0.28191499226502426
    #n_estimators = 91 ou 97
    #subsample = 0.7027664203395149
    
    mdl_gradient = GradientBoostingClassifier(random_state=43, learning_rate=0.28191499226502426, subsample=subsample)
    mdl_gradient.fit(X_train, y_train)
    
    p2 = mdl_gradient.predict(X_test) #.astype('int32')
    
    # Queremos minimizar o auc score
    return -mdl_gradient.score(X_test, y_test)

# Definindo espaço de busca
space_gradient = [#(0.01, 0.1)] #learning rate
         #(50, 200)] # n_estimators
         (0.01, 1)] # subsample
         #(2, 30)] # min_sample_split
         #(100, 200)] #max_features
         #(0.0, 0.05)] #min_weight
         #(3, 50)] #max_depth
         #(1,50)] # rdm_state

# fazendo o fit do modelo com 30 calls
resultado_dummy_gradient = dummy_minimize(treinar_modelo_gradient, space_gradient, random_state=1, verbose=1, n_calls=30)
resultado_dummy_gradient

In [ ]:
resultado_gp_gradient = gp_minimize(treinar_modelo_gradient, space_gradient, random_state=1, verbose=1, n_calls=30, n_random_starts=10)
resultado_gp_gradient

In [ ]:
from skopt.plots import plot_convergence
plot_convergence(resultado_dummy_gradient, resultado_gp_gradient)

In [ ]:
#BEST MODEL GRADIENT BOOSTING

model_GBoost = GradientBoostingRegressor(random_state=1, learning_rate=0.400231532805698, n_estimators=100,
                                            subsample=0.8478478075191571, min_samples_split=8, max_depth=3).fit(X_train, y_train)
model_GBoost.score(X_test, y_test)

# TUNANDO O LGBM

In [ ]:
def treinar_modelo_lgbm(params):
    #random_state = params[0]
    #learning_rate = params[0]
    #num_leaves = params[0]
    #min_child_samples = params[0]
    subsample = params[0]
    #colsample_bytree = params[0]
    
    print(params, '\n')
    
    #learning_rate=learning_rate, num_leaves=num_leaves, min_child_samples=min_child_samples,
                        #subsample=subsample, colsample_bytree=colsample_bytree, random_state=0, subsample_freq=1, 
                         #n_estimators=100
    
    mdl_lgbm = LGBMClassifier(random_state=0, learning_rate=0.6754657915377302, num_leaves=31, 
                              min_child_samples=20, subsample=subsample)
    mdl_lgbm.fit(X_train, y_train)
    
    p = mdl_lgbm.predict_proba(X_test)[:,1]
    
    # Queremos minimizar o auc score
    return -mdl_lgbm.score(X_test, y_test)

# Definindo nosso espaço de busca randômica. Não são tuplas, são ranges!
space_lgbm = [#(1, 60)] #learning rate
         #(0.1, 1.0, 'log-uniform')] #learning rate
         #(2, 64)] # num_leaves
         #(20, 64)] # min_child_samples
         (0.05, 1.0)] # subsample
         #(0.1, 1.0)] # colsample bytree

# fazendo o fit do modelo com 30 calls, ou seja, amostragens
resultado = dummy_minimize(treinar_modelo_lgbm, space_lgbm, random_state=1, verbose=1, n_calls=30)

In [ ]:
resultado_gp_xgb = gp_minimize(treinar_modelo_lgbm, space_lgbm, random_state=1, verbose=1, n_calls=30, n_random_starts=10)
resultado_gp_xgb

In [ ]:
#MELHOR RESULTADO XGBOOST
model_XGBoost = xgb.XGBRegressor(min_child_weight=1, learning_rate=0.28887118156158087, colsample_bytree=0.20921998968669897,
                              max_depth=62, subsample=1).fit(X_train, y_train)
model_XGBoost.score(X_test, y_test)

In [ ]:
pred1 = model_GBoost.predict(X_test)
df_pred1 = pd.DataFrame(pred1)
df_pred1

In [ ]:
df_pred1 = df_pred1.rename(columns={0: "Valor Predito"})
df_pred1['Valor Real'] = y_test

In [ ]:
df_ytest = pd.DataFrame(y_test)
df_ytest = df_ytest.reset_index()
df_ytest = df_ytest.drop(columns=['index'])

In [4]:
df_pred1['Valor Real'] = df_ytest['MdiaMs']

NameError: name 'df_ytest' is not defined

In [5]:
df_pred1['Diff1'] = df_pred1['Valor Predito']-df_pred1['Valor Real']
df_pred1['Diff2'] = df_pred1['Valor Real']-df_pred1['Valor Predito']
df_pred1

NameError: name 'df_pred1' is not defined

In [6]:
df_pred1['ConfDown'] = -df_pred1['Diff1'].std()*2
df_pred1['ConfUp'] = df_pred1['Diff1'].std()*2
df_pred1

NameError: name 'df_pred1' is not defined

In [ ]:
df_pred1['Diff1'].value_counts().sort_values()

In [ ]:
df_pred1[['ConfDown','Diff1','ConfUp']].plot.hist(bins=100)

In [ ]:
df_pred1['Diff1'].plot.hist(bins=100)

# UTILIZANDO O DATAFRAME COM TODAS AS VARIÁVEIS

In [ ]:
#UTILIZANDO O DATAFRAME COM TODAS AS VARIÁVEIS
df_rev_work2

In [ ]:
df_rev_work2.info()

In [ ]:
df_encoded_full = pd.get_dummies(df_rev_work2)

In [ ]:
df_encoded_full

In [ ]:
df_encoded_final = df_encoded_full.drop(columns=['Código IBGE', 'Valor Pedido', 'Mês'])

In [ ]:
cols=pd.Series(df_encoded_final.columns)
for dup in df_encoded_final.columns[df_encoded_final.columns.duplicated(keep=False)]: 
    cols[df_encoded_final.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(df_encoded_final.columns.get_loc(dup).sum())]
                                    )
df_encoded_final.columns=cols

In [ ]:
df_encoded_final2 = df_encoded_final.loc[:,~df_encoded_final.columns.duplicated()]

In [ ]:
df_encoded_final2 = df_encoded_final2.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [ ]:
df_encoded_final2 = df_encoded_final2.loc[:,~df_encoded_final2.columns.duplicated()]

In [ ]:
df_encoded_final2 = df_encoded_final2[df_encoded_final2['MdiaMs'] != 0]
df_encoded_final2['MdiaMs'].value_counts()

In [ ]:
X_full = df_encoded_final2.drop(columns=['MdiaMs'])
y_full = df_encoded_final2['MdiaMs'] #.astype('float64')

In [ ]:
y_full.plot.hist()

In [ ]:
y_full_log = np.log(y_full)
y_full_log.plot.hist()

In [ ]:
X_full_train, X_full_test, y_full_train, y_full_test = train_test_split(X_full, y_full, test_size=0.33, random_state=42)

In [ ]:
X_full_train, X_full_test, y_full_log_train, y_full_log_test = train_test_split(X, y_full_log, test_size=0.33, random_state=42)

In [ ]:
#comparando os modelos para verificar qual utilizar
#isso demorou MUITO pra rodar
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn import metrics

# criando uma lista com todos os modelos
classifiers = [
    #KNeighborsClassifier(3),
    #GaussianNB(),
    #LogisticRegression(),
    #SVC(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    LGBMRegressor(),
    xgb.XGBRegressor()]

In [ ]:
for clf in classifiers:
    # ajustando o modelo
    clf.fit(X_full_train, y_full_train)
    # armazenando o nome do modelo
    name = clf.__class__.__name__
    # imprimindo o nome do modelo
    print("="*30)
    print(name)
    # imprimindo os resultados
    print('****Results****')
    # fazendo predições
    # calculando as métricas
    y_pred = clf.predict(X_full_test)
    # imprimindo as métricas
    print("Score:", clf.score(X_full_test, y_full_test))
    #print("Precision:", metrics.precision_score(y_test, y_pred))
    #print("Recall:", metrics.recall_score(y_test, y_pred))

In [ ]:
def treinar_modelo_xgb(params):
    random_state = params[0]
    #eta = params[0]
    #min_child_weight = params[0]
    #max_depth = params[0]
    #max_leaves = params[0]
    #gamma = params[4]
    #subsample = params[5]
    #colsample_bytree = params[6]
    
    
    print(params, '\n')
    
    #learning_rate=lrn_rate, n_estimators=num_estim, subsample=subsample, min_samples_split=min_samp_split, 
    #                                     min_samples_leaf=min_samp_leaf, min_weight_fraction_leaf=min_weight, max_depth=max_depth
    
    mdl_xgb = GradientBoostingRegressor(random_state=15, )
    mdl_xgb.fit(X_full_train, y_full_train)
    
    p2_xgb = mdl_xgb.predict(X_full_test) #.astype('int32')
    
    # Queremos minimizar o auc score
    return -mdl_xgb.score(X_full_test, y_full_test)

# Definindo espaço de busca
space = [(1, 50)] #random_state
         #(0.01, 0.5)] #eta
         #(0, 1)] # min_child_weight
         #(1, 6)] # max_depth
         #(50, 150)] # max_leaves
         #(0,5), #gamma
         #(0, 1), #subsample
         #(0, 1)] #colsample_bytree

# fazendo o fit do modelo com 30 calls
resultado_dummy_xgb = dummy_minimize(treinar_modelo_xgb, space, random_state=1, verbose=1, n_calls=30)
resultado_dummy_xgb

In [ ]:
mdl_xgb.predict(X_test)

x_ax = range(len(y_full_test))
plt.plot(x_ax, y_full_test, label="original")
plt.plot(x_ax, y_full_pred, label="predicted")
plt.title("Boston test and predicted data")
plt.legend()
plt.show()

In [ ]:
from skopt.plots import plot_convergence
plot_convergence(resultado_dummy_xgb, resultado_gp_xgb)

In [ ]:
def treinar_modelo_gradient_full(params):
    #random_state = params[0]
    #learning_rate = params[0]
    n_estimators = params[0]
    #subsample = params[0]
    #min_samples_split = params[0]
    #max_features = params[0]
    #min_weight_fraction_leaf = params[0]
    #max_depth = params[0]
    
    
    print(params, '\n')
    
    #learning_rate=lrn_rate, n_estimators=num_estim, subsample=subsample, min_samples_split=min_samp_split, 
    #                                     min_samples_leaf=min_samp_leaf, min_weight_fraction_leaf=min_weight, max_depth=max_depth
    
    mdl_gradient_full = GradientBoostingRegressor(random_state=15, learning_rate=0.27260830969150496, 
                                                  n_estimators=n_estimators)
    mdl_gradient_full.fit(X_full_train, y_full_train)
    
    p2 = mdl_gradient_full.predict(X_full_test) #.astype('int32')
    
    # Queremos minimizar o auc score
    return -mdl_gradient_full.score(X_full_test, y_full_test)

# Definindo espaço de busca
space_gradient_full = [#(0.09, 0.3)] #learning rate
         (50, 500)] # n_estimators
         #(0.01, 0.9)] # subsample
         #(2, 50)] # min_sample_split
         #(1, 1020, 10)] #max_features
         #(0, 0.3)] #min_weight
         #(20, 100)] #max_depth
         #(1,50)] # rdm_state

# fazendo o fit do modelo com 30 calls
resultado_dummy_gradient_full = dummy_minimize(treinar_modelo_gradient_full, space_gradient_full, random_state=1, verbose=1, n_calls=30)
resultado_dummy_gradient_full

In [ ]:
resultado_gp = gp_minimize(treinar_modelo_gradient_full, space_gradient_full, random_state=1, verbose=1, n_calls=30, n_random_starts=10)
resultado_gp

In [ ]:
from skopt.plots import plot_convergence
plot_convergence(resultado_dummy, resultado_gp)

In [ ]:
importances = mdl.feature_importances_
std = np.std([tree.feature_importances_ for tree in mdl.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()
